# (OpenCV - Chap7) 영역처리
> 영역처리
- toc: true
- branch: master
- badges: false
- comments: true
- author: dinonene
- categories: [python]

## 7.1.2 블러링
블러링은 영상에서 화소값이 급격하게 변하는 부분들을 감소시켜 점진적으로 변하게 함으로써 영상에서 전체적으로 부드러운 느낌이 나게 하는 기술이다. 교재에 따라서 스무딩(smoothing)이라 하기도 한다.

In [2]:
import numpy as np, cv2

In [3]:
# 회선수행 함수 - 행렬 처리 방식 (속도 면에서 유리)
def filter(image, mask):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.float32) # 회선 결과 저장 행렬
    
    ycenter, xcenter = rows//2, cols//2 # 마스크 중심 좌표
    
    for i in range(ycenter, rows - ycenter): # 입력 행렬 반복 순회
        for j in range(xcenter, cols - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1 # 관심 영역 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1 # 관심 영역 너비 범위
            roi = image[y1:y2, x1:x2].astype('float32') # 관심 영역 형변환
            tmp = cv2.multiply(roi, mask) # 회선 적용 - 원소 간 곱셈
            dst[i, j] = cv2.sumElems(tmp)[0] # 출력 화소 저장
    return dst # 자료형 변환하여 반환

In [4]:
# 회선 수행 함수 - 화소 직접 근접
def filter2(image, mask):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.float32)                 # 회선 결과 저장 행렬
    xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2  # 마스크 중심 좌표

    for i in range(ycenter, rows - ycenter):                  # 입력 행렬 반복 순회
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):                    # 마스크 원소 순회
                for v in range(mask.shape[1]):
                    y, x = i + u - ycenter , j + v - xcenter
                    sum += image[y, x] * mask[u, v]           # 회선 수식
            dst[i, j] = sum
    return dst

In [5]:
image = cv2.imread("./ghtop_images/chap07_images/filter_blur.jpg", cv2.IMREAD_GRAYSCALE)  # 영상 읽기
if image is None: raise Exception("영상파일 읽기 오류")

In [6]:
# 블러링 마스크 원소 지정     
data = [1/9, 1/9, 1/9,
        1/9, 1/9, 1/9,
        1/9, 1/9, 1/9]

# 마스크 행렬 생성
mask = np.array(data, np.float32).reshape(3, 3)
blur1 = filter(image, mask)                # 회선 수행 - 행렬 처리 방식
blur2 = filter2(image, mask)               # 회선 수행 - 화소 직접 접근
blur1 = blur1.astype('uint8')              # 행렬 표시위해 uint8형 변환
blur2 = cv2.convertScaleAbs(blur2)

In [7]:
cv2.imshow('image', image)
cv2.imshow('blur1', blur1)
cv2.imshow('blur2', blur2)

cv2.waitKey(0)
cv2.destroyAllWindows() # 모든 창 닫기

ref : https://pinkocto.github.io/BP2022/python/2022/10/15/opencv.html